In [15]:
import sys
sys.path.append('../')

In [16]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%autoreload 2
from analyses.metrics import *

In [18]:
generated_paths = {
    "QM9": "/Users/ameyad/Documents/spherical-harmonic-net/others/qm9_xyz",
    # "Symphony_v3_132k": "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_bessel_embedding_attempt3_radius_fixed/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=1320000/molecules",
    # "Symphony_v3_330k": "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_bessel_embedding_attempt3_radius_fixed/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=3300000/molecules",
    # "Symphony_v5_132k": "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_bessel_embedding_attempt5_lower_lr_0.05noise/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=1320000/molecules",
    # "Symphony_v5_249k": "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_bessel_embedding_attempt5_lower_lr_0.05noise/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=2490000/molecules",
    "Symphony": "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_bessel_embedding_attempt6_edm_splits/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=7530000/molecules",
    "EDM": "/Users/ameyad/Documents/spherical-harmonic-net/others/edm-resampled/samples_edm_xyz",
    "GSchNet": "/Users/ameyad/Documents/spherical-harmonic-net/others/gschnet-edm-retrained/molecules",
    "GSphereNet": "/Users/ameyad/Documents/spherical-harmonic-net/others/gspherenet/molecules",
}

Corresponding molecules in OpenBabel:

In [19]:
all_generated_molecules_openbabel = {
    model: get_all_molecules_with_openbabel(path) for model, path in generated_paths.items()
}

valid_molecules_openbabel = {
    model: get_all_valid_molecules_with_openbabel(molecules) for model, molecules in all_generated_molecules_openbabel.items()
}

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is [/home/surajit/SURE/Arghya/publish/Data/data_workflows-master-3c7a0808d1ac395ecf485adb2c1f59b1a61b30fc/qm9_g4mp2/tmp/extracted/output/dsgdb9nsd_127200.log: Geometry 2])

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is [/home/surajit/SURE/Arghya/publish/Data/data_workflows-master-3c7a0808d1ac395ecf485adb2c1f59b1a61b30fc/qm9_g4mp2/tmp/extracted/output/dsgdb9nsd_125371.log: Geometry 3])

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is [/home/surajit/SURE/Arghya/publish/Data/data_workflows-master-3c7a0808d1ac395ecf485adb2c1f59b1a61b30fc/qm9_g4mp2/tmp/extracted/output/dsgdb9nsd_025403.log: Geometry 2])

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is [/home

Validity (OpenBabel):

In [20]:
# Make a dataframe for each model for the validity of the molecules.
validity_openbabel_df = pd.DataFrame(columns=["model", "validity"])

for model, molecules in valid_molecules_openbabel.items():
    validity_fraction = compute_validity(all_generated_molecules_openbabel[model], molecules)    
    validity_openbabel_df = pd.concat(
        [
            validity_openbabel_df,
            pd.DataFrame.from_records(
                {"model": model, "validity": [validity_fraction]}
            ),
        ],
        ignore_index=True,
    )

formatted_validity_df = validity_openbabel_df.transpose()
formatted_validity_df.columns = formatted_validity_df.iloc[0]
formatted_validity_df *= 100
formatted_validity_df = formatted_validity_df.drop("model")
formatted_validity_df.index = ["Validity via OpenBabel"]
formatted_validity_df = formatted_validity_df.astype(float)
formatted_validity_df = formatted_validity_df.applymap(lambda x: f"{x:.2f}")
print(formatted_validity_df.style.to_latex())

\begin{tabular}{llllll}
model & QM9 & Symphony & EDM & GSchNet & GSphereNet \\
Validity via OpenBabel & 94.60 & 74.69 & 77.75 & 61.83 & 9.86 \\
\end{tabular}



/var/folders/rf/gl_bs24d2sg1ptcd__f19_pw0000gn/T/ipykernel_35081/2721582254.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  validity_openbabel_df = pd.concat(
/var/folders/rf/gl_bs24d2sg1ptcd__f19_pw0000gn/T/ipykernel_35081/2721582254.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_validity_df = formatted_validity_df.applymap(lambda x: f"{x:.2f}")


In [21]:
# Make a dataframe for each model for the uniqueness of the molecules.
uniqueness_openbabel_df = pd.DataFrame(columns=["model", "uniqueness"])

for model, molecules in valid_molecules_openbabel.items():
    uniqueness_fraction = compute_uniqueness_with_openbabel(molecules)    
    uniqueness_openbabel_df = pd.concat(
        [
            uniqueness_openbabel_df,
            pd.DataFrame.from_records(
                {"model": model, "uniqueness": [uniqueness_fraction]}
            ),
        ],
        ignore_index=True,
    )

formatted_uniqueness_df = uniqueness_openbabel_df.transpose()
formatted_uniqueness_df.columns = formatted_uniqueness_df.iloc[0]
formatted_uniqueness_df *= 100
formatted_uniqueness_df = formatted_uniqueness_df.drop("model")
formatted_uniqueness_df = formatted_uniqueness_df.astype(float)
formatted_uniqueness_df.index = ["Uniqueness via OpenBabel"]
formatted_uniqueness_df = formatted_uniqueness_df.round(2)
formatted_uniqueness_df = formatted_uniqueness_df.applymap(lambda x: f"{x:.2f}")
print(formatted_uniqueness_df.style.to_latex())

\begin{tabular}{llllll}
model & QM9 & Symphony & EDM & GSchNet & GSphereNet \\
Uniqueness via OpenBabel & 99.97 & 99.61 & 99.95 & 98.71 & 7.51 \\
\end{tabular}



/var/folders/rf/gl_bs24d2sg1ptcd__f19_pw0000gn/T/ipykernel_35081/1566800241.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  uniqueness_openbabel_df = pd.concat(
/var/folders/rf/gl_bs24d2sg1ptcd__f19_pw0000gn/T/ipykernel_35081/1566800241.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_uniqueness_df = formatted_uniqueness_df.applymap(lambda x: f"{x:.2f}")
